## Cài đặt thư viện

In [1]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import os
from dotenv import load_dotenv
from RAG import DocumentStore, RAGPipeline
from ChunkDocument import DocumentProcessor
from WordReader import WordReader
import warnings
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

## Đọc dữ liệu từ file Excel

In [38]:
# Đọc file Excel
file_path = 'doc/EvaluateData.xlsx'  # Đường dẫn đến file Excel của bạn

# Đảm bảo đọc đúng định dạng
try:
    data = pd.read_excel(file_path, engine='openpyxl')
    questions = data['QUESTION'].tolist()
    correct_answers = data['ANSWER'].tolist()
except Exception as e:
    print(f"Lỗi khi đọc file Excel: {e}")

## Sinh ra câu trả lời và chunk từ mô hình RAG

In [39]:

# Hàm để sinh ra các chunk và câu trả lời RAG
def update_excel_with_chunks_and_answers(file_path, questions, correct_answers):
    warnings.filterwarnings("ignore", category=FutureWarning)
    load_dotenv()
    
    # Đọc dữ liệu từ file Excel cũ
    try:
        df_existing = pd.read_excel(file_path, engine='openpyxl')
    except Exception as e:
        print(f"Lỗi khi đọc file Excel: {e}")
        return

    # Đọc tài liệu và chuyển đổi chúng thành danh sách các phần văn bản
    file_paths = ["doc/So tay SV HCMUTE-2018.docx"]
    word_reader = WordReader(file_paths)

    # Đọc và xử lý các tài liệu từ WordReader
    sections = word_reader.process_documents()

    # Tạo DocumentProcessor và xử lý các tài liệu
    processor = DocumentProcessor(sections)
    documents = processor.process_documents()

    # Khởi tạo DocumentStore
    store = DocumentStore(documents)

    # Tạo một đối tượng RAGPipeline
    OPENAI_KEY = os.getenv('OPENAI_KEY')
    api_key = OPENAI_KEY
    rag_pipeline = RAGPipeline(documents=store, api_key=api_key, model_name='gpt-3.5-turbo')

    # Tạo DataFrame mới để lưu chunk và câu trả lời RAG
    new_data = {
        'CHUNK_1': [],
        'CHUNK_2': [],
        'CHUNK_3': [],
        'CHUNK_4': [],
        'RAG_ANSWER': []
    }

    # Lặp qua từng câu hỏi để sinh chunk và câu trả lời RAG
    for question in questions:
        retriever = store.get_retriever()  # Lấy retriever từ DocumentStore
        results = retriever.invoke(question)  # Truy xuất tài liệu liên quan

        # Lưu chunk vào danh sách, chỉ lấy 4 chunk đầu tiên
        chunks = [doc.page_content for doc in results][:4]
        while len(chunks) < 4:  # Đảm bảo có đủ 4 chunk, nếu không thêm None
            chunks.append(None)
        
        # Sinh câu trả lời từ mô hình RAG
        generated_answer = rag_pipeline.answer_question(question)

        # Thêm vào dữ liệu mới
        new_data['CHUNK_1'].append(chunks[0])
        new_data['CHUNK_2'].append(chunks[1])
        new_data['CHUNK_3'].append(chunks[2])
        new_data['CHUNK_4'].append(chunks[3])
        new_data['RAG_ANSWER'].append(generated_answer)

    # Tạo DataFrame từ dữ liệu mới
    df_new_chunks = pd.DataFrame(new_data)

    # Kết hợp DataFrame hiện tại và DataFrame mới
    df_updated = pd.concat([df_existing, df_new_chunks], axis=1)

    # Lưu DataFrame đã cập nhật vào file Excel cũ
    df_updated.to_excel(file_path, index=False, engine='openpyxl')



In [40]:
# Cập nhật Excel với các chunk và câu trả lời RAG
update_excel_with_chunks_and_answers(file_path, questions, correct_answers)

Chunks:
Chunk 1:
tập lớn. 
Điều 22. Cách tính điểm đánh giá bộ phận, điểm học phần  
1. Điểm đánh giá bộ phận và điểm thi kết thúc học phần được chấm theo thang điểm 10 (từ 0  đến 10), làm tròn đến một chữ số thập phân.  
2. Điểm học phần là tổng điểm của tất cả các điểm đánh giá bộ phận của học phần nhân với  trọng số tương ứng. Điểm học phần làm tròn đến một chữ số thập phân, sau đó được chuyển thành  điểm chữ như sau: 
a) Loại đạt: A (8,5 - 10) Giỏi 
B (7,0 - 8,4) Khá 
C (5,5 - 6,9) Trung bình 
D (4,0 - 5,4) Trung bình yếu 
b) Loại không đạt: F (dưới 4,0) Kém 
c) Đối với những học phần chưa đủ cơ sở để đưa vào tính điểm trung bình chung học kỳ, khi  xếp mức đánh giá được sử dụng các kí hiệu sau: 
I Chưa đủ dữ liệu đánh giá. 
X Chưa nhận được kết quả thi. 
d) Đối với những học phần được nhà trường cho phép chuyển điểm, khi xếp mức đánh giá  được sử dụng kí hiệu R viết kèm với kết quả. 
3. Việc xếp loại các mức điểm A, B, C, D, F được áp dụng cho các trường hợp sau đây: a) Đối với nhữ

## Định nghĩa các hàm

TF-IDF
- Vectorization: Sử dụng TfidfVectorizer để chuyển đổi các đoạn văn bản và câu trả lời thành một ma trận TF-IDF. Ma trận này thể hiện tần suất xuất hiện của từ trong các đoạn văn bản và khả năng xuất hiện của chúng trong toàn bộ tập văn bản.
- Tính độ tương đồng: Sử dụng hàm cosine_similarity để tính độ tương đồng cosine giữa các đoạn văn bản (chunks) và câu trả lời đúng. Độ tương đồng cosine đo lường mức độ tương đồng giữa hai vector (tại đây là vector TF-IDF) dựa trên góc giữa chúng.

BERT
- Mô hình BERT: Sử dụng SentenceTransformer với mô hình 'distilbert-base-nli-stsb-mean-tokens' để tạo ra các embedding cho từng đoạn văn bản và câu trả lời đúng.
- Embedding: Mỗi đoạn văn bản (chunk) và câu trả lời được chuyển đổi thành một vector embedding, đại diện cho ngữ nghĩa của chúng.
- Tính độ tương đồng: Sử dụng hàm pytorch_cos_sim để tính độ tương đồng cosine giữa các vector embedding của các đoạn văn bản và câu trả lời đúng.

In [20]:
# Hàm tiền xử lý văn bản
def preprocess_text(text):
    # Chuyển về chữ thường
    text = text.lower()
    return text

# Hàm tính độ tương đồng sử dụng TF-IDF
def calculate_tfidf_similarity(chunks, correct_answer):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(chunks + [correct_answer])
    
    # Tính cosine similarity
    similarity_matrix = cosine_similarity(tfidf_matrix[:-1], tfidf_matrix[-1:])
    return similarity_matrix.flatten()

# Hàm tính độ tương đồng sử dụng BERT
def calculate_bert_similarity(chunks, correct_answer):
    model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
    
    # Chuyển đổi chunks và correct_answer thành embeddings
    chunk_embeddings = model.encode(chunks)
    correct_answer_embedding = model.encode(correct_answer)

    # Tính cosine similarity
    cosine_scores = util.pytorch_cos_sim(chunk_embeddings, correct_answer_embedding)
    return cosine_scores.flatten()


## Đánh giá các chunk từ file Excel

In [27]:
# Hàm đánh giá các chunk và lưu kết quả vào file Excel
def evaluate_chunks(file_path):
    # Đọc dữ liệu từ file Excel
    try:
        df = pd.read_excel(file_path, engine='openpyxl')
    except Exception as e:
        print(f"Lỗi khi đọc file Excel: {e}")
        return

    # Lấy câu hỏi, câu trả lời đúng và các chunk
    questions = df['QUESTION'].tolist()
    correct_answers = df['ANSWER'].tolist()
    retrieved_chunks = df[['CHUNK_1', 'CHUNK_2', 'CHUNK_3', 'CHUNK_4']].values.tolist()

    # List để lưu trữ thông tin đánh giá từng chunk
    chunk_metrics = []
    
    total_precision_tfidf = 0
    total_recall_tfidf = 0
    total_f1_tfidf = 0
    total_precision_bert = 0
    total_recall_bert = 0
    total_f1_bert = 0
    total_exact_matches = 0

    total_questions = len(questions)

    # Duyệt từng câu hỏi
    for i in range(total_questions):
        correct_answer = preprocess_text(correct_answers[i])
        chunks = [preprocess_text(chunk) for chunk in retrieved_chunks[i]]

        # Tính độ tương đồng với TF-IDF
        tfidf_scores = calculate_tfidf_similarity(chunks, correct_answer)

        # Tính độ tương đồng với BERT
        bert_scores = calculate_bert_similarity(chunks, correct_answer)

        correct_count_tfidf = 0
        correct_count_bert = 0
        retrieved_count = len(chunks)

        # Duyệt qua từng chunk để ghi lại thông số
        for j, chunk in enumerate(chunks):
            tfidf_score = tfidf_scores[j]
            bert_score = bert_scores[j]

            # Kiểm tra chunk có chứa câu trả lời đúng không (ngưỡng 0.5)
            tfidf_match = tfidf_score > 0.3
            bert_match = bert_score > 0.3

            # Cộng tổng số chunk đúng (match)
            if tfidf_match:
                correct_count_tfidf += 1
            if bert_match:
                correct_count_bert += 1

            # Thêm thông tin đánh giá của chunk vào danh sách
            chunk_metrics.append({
                'Question': questions[i],
                'Correct Answer': correct_answers[i],
                'Chunk': chunk,
                'TF-IDF Score': tfidf_score,
                'BERT Score': bert_score,
                'TF-IDF Match': tfidf_match,
                'BERT Match': bert_match
            })

        # Tính Precision, Recall và F1 Score cho từng câu hỏi
        precision_tfidf = correct_count_tfidf / retrieved_count if retrieved_count > 0 else 0
        recall_tfidf = correct_count_tfidf / 1  # Chỉ có 1 câu trả lời đúng
        f1_tfidf = (2 * precision_tfidf * recall_tfidf) / (precision_tfidf + recall_tfidf) if (precision_tfidf + recall_tfidf) > 0 else 0
        
        precision_bert = correct_count_bert / retrieved_count if retrieved_count > 0 else 0
        recall_bert = correct_count_bert / 1  # Chỉ có 1 câu trả lời đúng
        f1_bert = (2 * precision_bert * recall_bert) / (precision_bert + recall_bert) if (precision_bert + recall_bert) > 0 else 0

        # Cộng tổng để tính trung bình sau này
        total_precision_tfidf += precision_tfidf
        total_recall_tfidf += recall_tfidf
        total_f1_tfidf += f1_tfidf

        total_precision_bert += precision_bert
        total_recall_bert += recall_bert
        total_f1_bert += f1_bert

        # Kiểm tra Exact Match
        if any(correct_answer in chunk for chunk in chunks):
            total_exact_matches += 1

    # Tính toán trung bình cho từng chỉ số
    avg_precision_tfidf = total_precision_tfidf / total_questions
    avg_recall_tfidf = total_recall_tfidf / total_questions
    avg_f1_tfidf = total_f1_tfidf / total_questions
    avg_precision_bert = total_precision_bert / total_questions
    avg_recall_bert = total_recall_bert / total_questions
    avg_f1_bert = total_f1_bert / total_questions
    exact_match_ratio = total_exact_matches / total_questions

    # Tạo DataFrame để lưu kết quả trung bình
    results_df = pd.DataFrame({
        'Metric': [
            'Precision (TF-IDF)', 
            'Recall (TF-IDF)', 
            'F1 Score (TF-IDF)', 
            'Precision (BERT)', 
            'Recall (BERT)', 
            'F1 Score (BERT)', 
            'Exact Match Ratio'
        ],
        'Value': [
            avg_precision_tfidf, 
            avg_recall_tfidf, 
            avg_f1_tfidf, 
            avg_precision_bert, 
            avg_recall_bert, 
            avg_f1_bert, 
            exact_match_ratio
        ]
    })

    # Lưu thông số đánh giá từng chunk vào file Excel
    chunk_metrics_df = pd.DataFrame(chunk_metrics)

    # Lưu các kết quả trung bình vào file Excel
    results_df.to_excel('average_evaluation_results.xlsx', index=False)

    # In thông số đánh giá ra màn hình
    print("\nKết quả đánh giá trung bình:")
    print(results_df)



In [28]:
file_path = 'doc/EvaluateData.xlsx'  # Đường dẫn đến file Excel của bạn
evaluate_chunks(file_path)

Thông số đánh giá từng chunk đã được lưu vào file text:

Kết quả đánh giá trung bình:
               Metric  Value
0  Precision (TF-IDF)  0.280
1     Recall (TF-IDF)  1.120
2   F1 Score (TF-IDF)  0.448
3    Precision (BERT)  1.000
4       Recall (BERT)  4.000
5     F1 Score (BERT)  1.600
6   Exact Match Ratio  0.000


### Đánh giá TF-IDF:
- Precision (TF-IDF): 0.280
    Trong các chunk trả về, khoảng 28% là chính xác khi sử dụng phương pháp TF-IDF
- F1 Score (TF-IDF): 0.448
    Đây là trung bình điều hòa giữa Precision và Recall cho TF-IDF, phản ánh rằng TF-IDF có độ chính xác và khả năng tìm kiếm khá tốt nhưng vẫn cần cải thiện.


### Đánh giá BERT:
- Precision (BERT): 1.000
    Độ chính xác của BERT là tuyệt đối, nghĩa là các chunk đúng mà BERT tìm thấy đều đúng.
- F1 Score (BERT): 1.600
    Với độ chính xác cao và recall cao, F1 score của BERT cũng rất tốt.
    
### Exact Match Ratio (Tỉ lệ câu trả lời chính xác hoàn toàn):
- Exact Match: 0.000 (0%) - Không có chunk nào hoàn toàn khớp với câu trả lời đúng, nghĩa là có thể chunk trả về chỉ chứa một phần nên không đủ chính xác để coi là "khớp hoàn toàn".


## Tính MAP

In [36]:
# Hàm tính MAP
def calculate_map_tfidf(chunk_metrics_df, questions, correct_answers):
    average_precisions = []
    """
    Tính toán Mean Average Precision (MAP) cho các chunk.

    Parameters:
    - chunk_metrics_df: DataFrame chứa thông tin đánh giá các chunk.
    - questions: Danh sách các câu hỏi.
    - correct_answers: Danh sách các câu trả lời đúng tương ứng.

    Returns:
    - map_value: Giá trị MAP tính được.
    """
    for question in questions:
        relevant_matches = chunk_metrics_df[chunk_metrics_df['Question'] == question]['TF-IDF Match'].values
        
        relevant_count = 0
        precision_at_k = []
        
        for k in range(len(relevant_matches)):
            if relevant_matches[k] == 1:
                relevant_count += 1
                precision_at_k.append(relevant_count / (k + 1))
        
        if relevant_count > 0:
            average_precisions.append(np.mean(precision_at_k))
        else:
            average_precisions.append(0.0)

    map_value = np.mean(average_precisions) if average_precisions else 0.0
    return map_value

In [37]:
# Hàm tính MAP
def calculate_map_bert(chunk_metrics_df, questions, correct_answers):
    average_precisions = []
    """
    Tính toán Mean Average Precision (MAP) cho các chunk.

    Parameters:
    - chunk_metrics_df: DataFrame chứa thông tin đánh giá các chunk.
    - questions: Danh sách các câu hỏi.
    - correct_answers: Danh sách các câu trả lời đúng tương ứng.

    Returns:
    - map_value: Giá trị MAP tính được.
    """
    for question in questions:
        relevant_matches = chunk_metrics_df[chunk_metrics_df['Question'] == question]['BERT Match'].values
        
        relevant_count = 0
        precision_at_k = []
        
        for k in range(len(relevant_matches)):
            if relevant_matches[k] == 1:
                relevant_count += 1
                precision_at_k.append(relevant_count / (k + 1))
        
        if relevant_count > 0:
            average_precisions.append(np.mean(precision_at_k))
        else:
            average_precisions.append(0.0)

    map_value = np.mean(average_precisions) if average_precisions else 0.0
    return map_value

In [39]:
# Hàm đánh giá các chunk
def evaluate_map_chunks(file_path):
    # Đọc dữ liệu từ file Excel
    try:
        df = pd.read_excel(file_path, engine='openpyxl')
    except Exception as e:
        print(f"Lỗi khi đọc file Excel: {e}")
        return

    # Lấy câu hỏi, câu trả lời đúng và các chunk
    questions = df['QUESTION'].tolist()
    correct_answers = df['ANSWER'].tolist()
    retrieved_chunks = df[['CHUNK_1', 'CHUNK_2', 'CHUNK_3', 'CHUNK_4']].values.tolist()

    total_questions = len(questions)
    
    # Tạo danh sách để lưu thông tin chunk
    chunk_metrics = []
    
    # Duyệt từng câu hỏi
    for i in range(total_questions):
        correct_answer = preprocess_text(correct_answers[i])
        chunks = [preprocess_text(chunk) for chunk in retrieved_chunks[i]]

        # Tính độ tương đồng với TF-IDF
        tfidf_scores = calculate_tfidf_similarity(chunks, correct_answer)

        # Tính độ tương đồng với BERT
        bert_scores = calculate_bert_similarity(chunks, correct_answer)

        # Duyệt qua từng chunk để ghi lại thông số
        for j, chunk in enumerate(chunks):
            tfidf_score = tfidf_scores[j]
            bert_score = bert_scores[j]

            # Kiểm tra chunk có chứa câu trả lời đúng không (ngưỡng 0.3)
            tfidf_match = tfidf_score > 0.3  # Ngưỡng có thể điều chỉnh
            bert_match = bert_score > 0.7

            # Thêm thông tin đánh giá của chunk vào danh sách
            chunk_metrics.append({
                'Question': questions[i],
                'Correct Answer': correct_answers[i],
                'Chunk': chunk,
                'TF-IDF Score': tfidf_score,
                'BERT Score': bert_score,
                'TF-IDF Match': 1 if tfidf_match else 0,
                'BERT Match': 1 if bert_match else 0
            })

    # Chuyển đổi danh sách chunk_metrics thành DataFrame
    chunk_metrics_df = pd.DataFrame(chunk_metrics)

    # Tính MAP cho TF-IDF và BERT
    map_tfidf = calculate_map_tfidf(chunk_metrics_df, questions, correct_answers)
    map_bert = calculate_map_bert(chunk_metrics_df, questions, correct_answers)

    # Lưu thông tin chunk vào file Excel
    chunk_metrics_df.to_excel('chunk_metrics.xlsx', index=False)

    # In kết quả MAP
    print(f'Mean Average Precision (TF-IDF): {map_tfidf:.3f}')
    print(f'Mean Average Precision (BERT): {map_bert:.3f}')

# Gọi hàm để thực hiện đánh giá
file_path = 'doc/EvaluateData.xlsx'  # Đường dẫn đến file Excel của bạn
evaluate_map_chunks(file_path)

Mean Average Precision (TF-IDF): 0.562
Mean Average Precision (BERT): 0.962
